In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import math
import requests
from io import BytesIO
from datetime import datetime

import nfl_data_py as nfl

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import plotly.colors as cl
from plotly.subplots import make_subplots

from PIL import Image

from resources.tier_chart import tier_chart
from resources.plotly_theme import nfl_template
from resources.get_nfl_data import get_team_info, get_pbp_data
from resources.team_stats import get_team_data

pio.templates['nfl_template'] = nfl_template

# Process Data

In [2]:
''' Params / Constants '''

## Params ##
THRU_WEEK = 8
EXPORT = False

## Constants ##

LEAGUE_LOGO = 'https://raw.githubusercontent.com/nflverse/nflverse-pbp/master/NFL.png'
PLOT_BUFFER = 0.05

In [3]:
''' Import Data '''

# Import
team_data = get_team_info()
pbp_data = get_pbp_data(years=[2025])

# Filter to current week
pbp_filtered = pbp_data.loc[pbp_data['week'] <= THRU_WEEK, :]

week_data = pbp_data.loc[pbp_data['week'] == THRU_WEEK, :].copy()

print(team_data.shape)
print(team_data.head().to_string())
print(pbp_filtered.shape)
print(pbp_filtered.head().to_string())
print(week_data.shape)
print(week_data.head().to_string())

2025 done.
Downcasting floats.
(32, 15)
                   team_name  team_id  team_nick team_conf team_division team_color team_color2 team_color3 team_color4                                                                                                   team_logo_wikipedia                                          team_logo_espn                                                          team_wordmark                                         team_conference_logo                                                        team_league_logo                                                          team_logo_squared
team_abbr                                                                                                                                                                                                                                                                                                                                                                                          

In [4]:
''' Get Stats '''

## Offense ##

# Overall
team_offense = get_team_data(pbp_filtered, unit='offense')
team_offense = team_offense.merge(team_data[['team_logo_espn', 'team_wordmark']], left_index=True, right_index=True)

print(team_offense.sort_values(by='EPA / Play', ascending=False).head().to_string())

# Week
week_offense = get_team_data(week_data, unit='offense')
week_offense = week_offense.merge(team_data[['team_logo_espn', 'team_wordmark']], left_index=True, right_index=True)

print(week_offense.sort_values(by='EPA / Play', ascending=False).head().to_string())


## Defense ##

# Overall
team_defense = get_team_data(pbp_filtered, unit='defense')
team_defense = team_defense.merge(team_data[['team_logo_espn', 'team_wordmark']], left_index=True, right_index=True)

print(team_defense.sort_values(by='EPA / Play', ascending=True).head().to_string())

# Week
week_defense = get_team_data(week_data, unit='defense')
week_defense = week_defense.merge(team_data[['team_logo_espn', 'team_wordmark']], left_index=True, right_index=True)

print(week_defense.sort_values(by='EPA / Play', ascending=True).head().to_string())

         Games  Plays   TDs  FirstDowns  RushAttempts  RushYards  RushTDs  Rush1Ds  DesignedRushPlays  DesignedRushAttempts  DesignedRushYards  QBScrambles  ScrambleYards  DesignedPassPlays  Dropbacks  PassCompletions  PassAttempts  PassYards  PassTDs  Pass1Ds  Sacks  SackYards  INTs  TFLs  Fumbles  Penalties  PenaltyYards  Penalty1Ds  Drives  Total Yards  Turnovers  PlaysAdv  PassPlays  RushPlays         EPA    RushEPA    PassEPA  Successes  RushSuccesses  PassSuccesses  EPA / Play  Rush EPA / Play  Pass EPA / Play  Success Rate  Rush Success Rate  Pass Success Rate                                     team_logo_espn                                                          team_wordmark
posteam                                                                                                                                                                                                                                                                                                         

## Rushing

In [22]:
''' Rushing Tiers '''


fig = tier_chart(
    data_frame=team_offense,
    y_col='Rush Success Rate',
    x_col='Rush EPA / Play',
    logos_col='team_logo_espn',
    n_tiers=7,
    title=f'NFL Rushing Tiers<br><sup>Thru Week {THRU_WEEK}</sup>'
)

fig.update_layout_images(sizex=0.03, sizey=0.03)
fig.update_yaxes(
    tickformat='.0%',
)
fig.update_xaxes(
    tickformat='.2f'
)
fig.update_layout(
    height=500, width=700
)
fig.show()

if EXPORT: pio.write_image(fig, f'Rushing Tiers - Thru Week {THRU_WEEK}.png', scale=6, width=700, height=500)

y = 0.32448x + 0.42963788520945884
x_range = 0.44599998
y_range = 0.144
slope = 0.3228699564933777
1
1


In [6]:
''' Rushing Profiles '''


fig = go.Figure()

## Data
X = team_offense['RushAttempts'].to_numpy()
Y = team_offense['Rush EPA / Play'].to_numpy()
LOGOS = team_offense['team_logo_espn'].to_numpy()

x_range = X.max() - X.min()
y_range = Y.max() - Y.min()

## Scatter
fig = px.scatter(
    x=X,
    y=Y,
)

# Logos
for i in range(len(X)):
    fig.add_layout_image(
        source=LOGOS[i],  # The loaded image
        xref="x",    # Reference x-coordinates to the x-axis
        yref="y",    # Reference y-coordinates to the y-axis
        x=X[i], # X-coordinate of the image's center
        y=Y[i], # Y-coordinate of the image's center
        sizex=5,   # Width of the image in data units
        sizey=5,   # Height of the image in data units
        xanchor="center", # Anchor the image by its center horizontally
        yanchor="middle", # Anchor the image by its middle vertically
        layer="above", # Place image above other plot elements
        opacity=0.9
    )

# Quadrant annotations
fig.add_annotation(
    text='RUSH A LOT<br>GOOD RUSHING TEAM',
    showarrow=False,
    xref='x',
    yref='y',
    y=Y.max() - (y_range*PLOT_BUFFER), 
    x=X.max() - (x_range*PLOT_BUFFER),
    align='center',
    font=dict(color="#646c76", weight='bold')
)
fig.add_annotation(
    text='RUSH A LOT<br>BAD RUSHING TEAM',
    showarrow=False,
    xref='x',
    yref='y',
    y=Y.min() + (y_range*PLOT_BUFFER), 
    x=X.max() - (x_range*PLOT_BUFFER),
    align='center',
    font=dict(color="#646c76", weight='bold')
)
fig.add_annotation(
    text='DON\'T RUSH A LOT<br>GOOD RUSHING TEAM',
    showarrow=False,
    xref='x',
    yref='y',
    y=Y.max() - (y_range*PLOT_BUFFER), 
    x=X.min() + (x_range*PLOT_BUFFER),
    align='center',
    # font=dict(color="#467bd0", weight='bold')
    font=dict(color="#646c76", weight='bold')
)
fig.add_annotation(
    text='DON\'T RUSH A LOT<br>BAD RUSHING TEAM',
    showarrow=False,
    xref='x',
    yref='y',
    y=Y.min() + (y_range*PLOT_BUFFER), 
    x=X.min() + (x_range*PLOT_BUFFER),
    align='center',
    font=dict(color="#646c76", weight='bold')
)
    
## Format
VIZ_X_RANGE = (X.min() - (x_range*PLOT_BUFFER), X.max() + (x_range*PLOT_BUFFER))
VIZ_Y_RANGE = (Y.min() - (y_range*PLOT_BUFFER), Y.max() + (y_range*PLOT_BUFFER))

fig.add_vline(x=X.mean(), line_width=1, line_dash="dash", line_color="#CB4747", layer='above')
fig.add_hline(y=Y.mean(), line_width=1, line_dash="dash", line_color="#CB4747", layer='above')

fig.update_traces(
    marker=dict(opacity=0)
)
fig.update_yaxes(
    title='EPA / Play',
    range=VIZ_Y_RANGE,
    tickformat='.2f'
)
fig.update_xaxes(
    title='Attempts',
    range=VIZ_X_RANGE,
    tickformat=','
)
fig.update_layout(
    template="nfl_template",
    title=dict(
        text=f'NFL Team Rushing Profiles<br><sup>Thru Week {THRU_WEEK}</sup>'
    ),
    showlegend=False,
    height=500, width=700
)

# Credits
fig.add_annotation(
    text=f'Figure: @jmillerr240 | Data: nfl_data_py | {datetime.today():%Y-%m-%d}',
    showarrow=False,
    xref='paper',
    yref='paper',
    y=-0.1, 
    x=1,
    align='left'
)
fig.show()

# Export
if EXPORT: pio.write_image(fig, f'Team Rushing Profiles - Thru Week {THRU_WEEK}.png', scale=6, width=700, height=500)

## Passing

In [21]:
''' Passing Tiers '''

fig = tier_chart(
    data_frame=team_offense,
    y_col='Pass Success Rate',
    x_col='Pass EPA / Play',
    logos_col='team_logo_espn',
    title=f'NFL Passing Tiers<br><sup>Thru Week {THRU_WEEK}</sup>',
    n_tiers=7,
)

fig.update_yaxes(
    tickformat='.0%'
)
fig.update_xaxes(
    tickformat='.2f'
)
fig.update_layout_images(sizex=0.04, sizey=0.04)
fig.update_layout(width=700, height=500)
fig.show()

# Export
if EXPORT: pio.write_image(fig, f'Passing Tiers - Thru Week {THRU_WEEK}.png', scale=6, width=700, height=500)

y = 0.26015x + 0.45011922949559297
x_range = 0.56700003
y_range = 0.19199997
slope = 0.3386242687702179
1
1


In [8]:
''' Passing Profile '''

fig = go.Figure()


## Data
X = team_offense['PassAttempts'].to_numpy()
Y = team_offense['Pass EPA / Play'].to_numpy()
LOGOS = team_offense['team_logo_espn'].to_numpy()

x_range = X.max() - X.min()
y_range = Y.max() - Y.min()

## Scatter
fig = px.scatter(
    x=X,
    y=Y,
)

# Logos
for i in range(len(X)):
    fig.add_layout_image(
        source=LOGOS[i],  # The loaded image
        xref="x",    # Reference x-coordinates to the x-axis
        yref="y",    # Reference y-coordinates to the y-axis
        x=X[i], # X-coordinate of the image's center
        y=Y[i], # Y-coordinate of the image's center
        sizex=8,   # Width of the image in data units
        sizey=8,   # Height of the image in data units
        xanchor="center", # Anchor the image by its center horizontally
        yanchor="middle", # Anchor the image by its middle vertically
        layer="above", # Place image above other plot elements
        opacity=0.9
    )

# Quadrant annotations
fig.add_annotation(
    # text='Pass a lot and are good at it',
    text='PASS A LOT<br>GOOD PASSING TEAM',
    showarrow=False,
    xref='x',
    yref='y',
    y=Y.max() - (y_range*PLOT_BUFFER), 
    x=X.max() - (x_range*PLOT_BUFFER),
    # y=Y.mean() + (y_range*0.4), 
    # x=X.mean() + (x_range*0.25),
    align='center',
    font=dict(color="#646c76", weight='bold')
)
fig.add_annotation(
    # text='Pass a lot but are bad at it',
    text='PASS A LOT<br>BAD PASSING TEAM',
    showarrow=False,
    xref='x',
    yref='y',
    # y=Y.mean() - (y_range*0.4), 
    # x=X.mean() + (x_range*0.25),
    y=Y.min() + (y_range*PLOT_BUFFER), 
    x=X.max() - (x_range*PLOT_BUFFER),
    align='center',
    font=dict(color="#646c76", weight='bold')
)
fig.add_annotation(
    # text='Don\'t pass a lot but are good at it',
    text='DON\'T PASS A LOT<br>GOOD PASSING TEAM',
    showarrow=False,
    xref='x',
    yref='y',
    y=Y.max() - (y_range*PLOT_BUFFER), 
    x=X.min() + (x_range*PLOT_BUFFER),
    # y=Y.mean() + (y_range*0.4), 
    # x=X.mean() - (x_range*0.25),
    align='center',
    # font=dict(color="#467bd0", weight='bold')
    font=dict(color="#646c76", weight='bold')
)
fig.add_annotation(
    # text='Don\'t pass a lot and are bad at it',
    text='DON\'T PASS A LOT<br>BAD PASSING TEAM',
    showarrow=False,
    xref='x',
    yref='y',
    y=Y.min() + (y_range*PLOT_BUFFER), 
    x=X.min() + (x_range*PLOT_BUFFER),
    # y=Y.mean() - (y_range*0.4), 
    # x=X.mean() - (x_range*0.25),
    align='center',
    font=dict(color="#646c76", weight='bold')
)

## Format
VIZ_X_RANGE = (X.min() - (x_range*PLOT_BUFFER), X.max() + (x_range*PLOT_BUFFER))
VIZ_Y_RANGE = (Y.min() - (y_range*PLOT_BUFFER), Y.max() + (y_range*PLOT_BUFFER))

fig.add_vline(x=X.mean(), line_width=1, line_dash="dash", line_color="#CB4747", layer='above')
fig.add_hline(y=Y.mean(), line_width=1, line_dash="dash", line_color="#CB4747", layer='above')

fig.update_traces(
    marker=dict(opacity=0)
)
fig.update_yaxes(
    title='EPA / Play',
    range=VIZ_Y_RANGE,
    tickformat='.2f'
)
fig.update_xaxes(
    title='Attempts',
    range=VIZ_X_RANGE,
    tickformat=','
)
fig.update_layout(
    template="nfl_template",
    title=dict(
        text=f'NFL Team Passing Profiles<br><sup>Thru Week {THRU_WEEK}</sup>'
    ),
    showlegend=False,
    width=700, height=500
)

# Credits
fig.add_annotation(
    text=f'Figure: @jmillerr240 | Data: nfl_data_py | {datetime.today():%Y-%m-%d}',
    showarrow=False,
    xref='paper',
    yref='paper',
    y=-0.1, 
    x=1,
    align='left'
)
fig.show()

# Export
if EXPORT: pio.write_image(fig, f'Team Passing Profiles - Thru Week {THRU_WEEK}.png', scale=6, width=700, height=500)

In [20]:
''' Offense Tiers '''

## Chart
fig = tier_chart(
    data_frame=team_offense,
    x_col='Pass EPA / Play',
    y_col='Rush EPA / Play',
    logos_col='team_logo_espn',
    title=f'NFL Offense Tiers<br><sup>Thru Week {THRU_WEEK}</sup>',
    n_tiers=7,
)

fig.update_xaxes(
    tickformat='0.2f'
)
fig.update_yaxes(
    tickformat='0.2f'
)
fig.update_layout_images(sizey=0.07)
fig.update_layout(width=700, height=500)
fig.update_annotations(y=-.1)

fig.show()

# Export
if EXPORT: pio.write_image(fig, f'Team Offense Tiers - Thru Week {THRU_WEEK}.png', scale=6, width=700, height=500)

y = 0.12839x + -0.07967753282049028
x_range = 0.56700003
y_range = 0.44599998
slope = 0.7865960597991943
1
1


In [ ]:
''' Offense Ranks '''

## Data ##
offense_ranks = team_offense.copy().rename(columns={
    'RushYards': 'Rush Yards',
    'PassYards': 'Pass Yards',
    'FirstDowns': 'First Downs',
    'Fumbles': 'Fumbles Lost',
})

# Rank
data_cols = ['Plays', 'Pass Yards', 'Rush Yards', 'First Downs', 'TDs', 'INTs', 'Fumbles Lost', 'Sacks', 'EPA / Play']
rank_cols = [f'{col}_rank' for col in data_cols]

for col in data_cols:
    if col in ['INTs', 'Fumbles', 'Sacks']:
        offense_ranks[f'{col}_rank'] = offense_ranks[col].rank(axis=0, method='max', ascending=True)
    else:
        offense_ranks[f'{col}_rank'] = offense_ranks[col].rank(axis=0, method='min', ascending=False)

offense_ranks = offense_ranks[data_cols + rank_cols]

# Logos
offense_ranks = offense_ranks.merge(team_data[['team_logo_espn', 'team_wordmark']], left_index=True, right_index=True)

print(offense_ranks.sort_values(by='EPA / Play', ascending=False).head().to_string())


## Viz ##

# Data
x = data_cols
y = offense_ranks.index.to_numpy()
z = offense_ranks[rank_cols].to_numpy()
text = offense_ranks[rank_cols].astype(int).astype(str).to_numpy()
wordmarks = offense_ranks['team_wordmark'].tolist()

# Heat map
heat_map = go.Heatmap(
    x=x, y=y, z=z, 
    text=text, texttemplate="%{text}", textfont=dict(size=10, color='white'),
    colorscale='PRGn', #['navy', '#c8c8c8', '#4bb543'], #'Cividis',
    reversescale=True,
    showscale=False,
    colorbar=dict(
        lenmode='fraction',
        len=0.5,
        title=dict(
            text='Rank',
            font=dict(weight='bold')
        ),
        xref='paper', yref='paper',
        x=1.1, xanchor='right'
    ),
    xgap=2, ygap=2
)

# Create figure
fig = go.Figure()

fig.add_trace(heat_map)

# Logos
for i in range(len(wordmarks)):
# for i in range(2):
    response = requests.get(wordmarks[i])
    logo_img = Image.open(BytesIO(response.content))
    fig.add_layout_image(
        x=-0.075,
        y=(1-(1/64))-((1/32)*i),
        sizex=.125,
        sizey=.125,
        xanchor='center',
        yanchor='middle',
        xref='paper', 
        yref='paper',
        source=logo_img,
    )
    
# Format
fig.update_xaxes(
    side='top',
    showgrid=False,
    ticks='',
    tickfont=dict(weight='bold'),
)
fig.update_yaxes(
    autorange='reversed',
    showgrid=False,
    ticks='',
    visible=False,
)
fig.update_layout(
    template='nfl_template',
    title=dict(
        text=f'NFL Offensive Ranks<br><sup>Thru Week {THRU_WEEK}</sup>',
        x=0.035
    ),
    margin=dict(t=100, b=25, l=100, r=25),
    height=900,
    width=700,
    margin_pad=5
)

# Credits
fig.add_annotation(
    text=f'Figure: @clankeranalytic | Data: nfl_data_py | {datetime.today():%Y-%m-%d}',
    xref='paper', yref='paper',
    x=1, y=-.025,
    showarrow=False,
    align='right'
)

fig.show()

# Export
if EXPORT: pio.write_image(fig, file=f'NFL Offensive Ranks - Thru Week {THRU_WEEK}.png', height=900, width=700, scale=6)

         Plays  Pass Yards  Rush Yards  First Downs   TDs  INTs  Fumbles Lost  Sacks  EPA / Play  Plays_rank  Pass Yards_rank  Rush Yards_rank  First Downs_rank  TDs_rank  INTs_rank  Fumbles Lost_rank  Sacks_rank  EPA / Play_rank                                     team_logo_espn                                                          team_wordmark
posteam                                                                                                                                                                                                                                                                                                                                                        
IND        471      2007.0      1072.0        184.0  31.0   3.0           1.0    9.0       0.231        14.0              5.0              6.0               2.0       1.0        9.0               27.0         3.0              1.0  https://a.espncdn.com/i/teamlogos/nfl/500/ind.png  https://github

In [20]:
''' EPA by Field Position '''

## Calc Data
# Filter out special teams
pbp_field_pos = pbp_filtered.loc[~pbp_filtered['Is Special Teams Play'],:].copy()

# Round yardlines
pbp_field_pos['yardline_rnd'] = np.ceil(pbp_field_pos['yardline_100'] / 5)*5

# Group by yardline
by_yardline = pbp_field_pos.groupby(['posteam', 'yardline_rnd']).aggregate(
    Plays=('posteam', 'size'),
    EPA=('epa', 'sum')
)
totals = by_yardline.groupby(['yardline_rnd']).aggregate(
    Plays=('Plays', 'sum'),
    EPA=('EPA', 'sum')
)
totals.index = pd.MultiIndex.from_product([['League'], totals.index])
by_yardline = pd.concat([by_yardline, totals])

print(by_yardline.loc[by_yardline.index.get_level_values(0)=='League'].to_string())


## Heat maps

MAX_EPA = by_yardline.loc[by_yardline.index.get_level_values(0) != 'League', 'EPA'].max()
MIN_EPA = by_yardline.loc[by_yardline.index.get_level_values(0) != 'League', 'EPA'].min()
print(MIN_EPA, MAX_EPA)

# Field
img = Image.open('./resources/field white.jpg')
img_width, img_height = img.size
ratio = img_width / img_height

# Yardlines
yardlines_raw = by_yardline.index.get_level_values(1).unique()
yardlines = np.where(yardlines_raw > 50, '-'+(50 - (yardlines_raw - 50)).astype(int).astype(str), '+'+yardlines_raw.astype(int).astype(str)).tolist()

# Divisions
divisions = team_data.reset_index().set_index(['team_division', 'team_abbr']).sort_index()
divisions = divisions.loc[divisions.index.get_level_values(1).isin(pbp_data['posteam'].unique()), :]
print(divisions.head().to_string())


## Create heatmaps
for division in divisions.index.get_level_values(0).unique():
    division_sl = divisions.loc[divisions.index.get_level_values(0) == division, :]
    teams = division_sl.index.get_level_values(1).sort_values().tolist()
    conf_logo = division_sl['team_conference_logo'].unique().tolist()[0]

    logos = []
    nick_names = []
    total_epas = []
    heat_maps = []
    for team in teams:
        # Team
        df = by_yardline.loc[by_yardline.index.get_level_values(0) == team, :].copy()
        
        # Logo
        team_info = team_data.loc[team_data.index == team, :]
        name = team_info['team_nick'].values[0]
        logo = team_info['team_logo_espn'].values[0]

        # Fill in missing yardlines
        for y in yardlines_raw:
            if y not in df.index.get_level_values(1):
                df.loc[(team, y), :] = [0, 0]
        df = df.sort_index(ascending=True)

        # Data inputs
        x = [team for i in range(len(yardlines))] #by_yardline.index.get_level_values(0)
        y = yardlines                       #y=colts_epa.index.get_level_values(1)
        z = df['EPA'].tolist()

        # Heatmap
        heat_map = go.Heatmap(
            x=y,
            y=x,
            z=z,
            opacity=0.7,
            # opacity=1,
            coloraxis='coloraxis',
            zmid=0
        )

        heat_maps.append(heat_map)
        total_epas.append(df['EPA'].sum())
        logos.append(logo)
        nick_names.append(name)

    N_COLS = 2
    N_ROWS = 2
    titles = []
    for i in range(len(teams)):
        title = f'{nick_names[i]} ({total_epas[i]:,.1f} Total EPA)'
        titles.append(title)

    ## Build Figure
    fig = make_subplots(rows=N_ROWS, cols=N_COLS, subplot_titles=titles)
    fig.update_annotations(font=dict(size=14))

    SIZEX = 0.45 #0.2885
    X_SPACING = (1 - (SIZEX*N_COLS)) / (N_COLS - 1)

    SIZEY = 0.375
    Y_SPACING = (1 - (SIZEY*N_ROWS)) / (N_ROWS - 1)

    i = 0
    for r in range(1, N_ROWS+1):
        for c in range(1, N_COLS+1):
            # Heat map
            fig.add_trace(
                heat_maps[i],
                row=r, col=c
            )

            # Field
            x = (SIZEX*(c-1)) + (X_SPACING*(c-1))
            y = 1 - ((SIZEY*(r-1)) + (Y_SPACING*(r-1)))
            fig.add_layout_image(
                x=x,
                y=y,
                sizey=SIZEY,
                sizex=SIZEX,
                xref="paper",
                yref="paper",
                opacity=1,
                layer="below",
                # opacity=0.2,
                # layer="above",
                sizing="stretch",
                source=img
            )

            # Logo in middle of field
            xref = 'x' if i == 0 else f'x{i+1}'
            yref = 'y' if i == 0 else f'y{i+1}'
            fig.add_layout_image(
                x=9.5,
                y=0,
                sizey=4,
                sizex=4,
                xref=xref,
                yref=yref,
                opacity=0.6,
                layer="above",
                sizing="contain",
                yanchor='middle',
                xanchor='center',
                source=logos[i]
            )

            i += 1

    # Conference Logo
    response = requests.get(conf_logo)
    conf_logo_img = Image.open(BytesIO(response.content))
    fig.add_layout_image(
        x=1,
        y=1.2,
        sizex=.15,
        sizey=.15,
        xanchor='right',
        yanchor='middle',
        xref='paper', 
        yref='paper',
        source=conf_logo_img,
    )

    # Direction annotation
    fig.add_annotation(
        text='<i>Moving left to right</i>',
        font=dict(size=12),
        xref='paper', yref='paper',
        x=0.5, y=0.5,
        showarrow=False
    )

    ## Formatting
    fig.update_xaxes(
        type='category',
        categoryorder='array', 
        categoryarray=yardlines,
        autorange='reversed',
        showgrid=False,
        visible=False,
    )
    fig.update_yaxes(
        showticklabels=False,
        showgrid=False,
        visible=False
    )

    r = max(abs(MIN_EPA), abs(MAX_EPA))
    fig.update_coloraxes(
        cmin=-1*r,
        cmax=r,
        cmid=0,
        colorscale=["#ae0d0a", 'white', "#297523"],
        colorbar_title=dict(text='Total EPA', font=dict(weight='bold'))
    )
    fig.update_layout(
        template='nfl_template',
        title=dict(
            text=f'Total <span style="color: #3f9938">EPA</span> by Field Position<br><sup>{division}</sup>'
        ),
        autosize=False,
        height=500,
        width=900,
    )

    # Credits
    fig.add_annotation(
        text=f'Thru Week {THRU_WEEK}<br>Figure: @clankeranalytic | Data: nfl_data_py | {datetime.today():%Y-%m-%d}',
        xref='paper', yref='paper',
        x=1, y=-.15,
        showarrow=False,
        align='right'
    )

    fig.show()

    # pio.write_image(fig, file=f'EPA by Field Position - {division} - Week {THRU_WEEK}.png', scale=6, height=500, width=900)



                      Plays        EPA
posteam yardline_rnd                  
League  5.0             587  76.951691
        10.0            472  40.455894
        15.0            574 -27.236822
        20.0            573  10.351196
        25.0            608  31.900066
        30.0            621  29.247730
        35.0            750   5.787171
        40.0            864 -58.831230
        45.0            869  73.357674
        50.0            870   9.038662
        55.0            925 -50.898186
        60.0           1086 -15.493578
        65.0           1290 -17.746586
        70.0           1122   2.383332
        75.0           1052 -24.104193
        80.0            772  82.466454
        85.0            445  35.218922
        90.0            346   4.810111
        95.0            206 -16.478577
        100.0            92 -18.453106
-32.124184 23.369986
                                    team_name  team_id team_nick team_conf team_color team_color2 team_color3 team_color4

# Defense

In [19]:
''' Rush D Tiers '''

fig = tier_chart(
    data_frame=team_defense,
    y_col='Rush Success Rate',
    x_col='Rush EPA / Play',
    logos_col='team_logo_espn',
    title=f'NFL Rushing Defense Tiers<br><sup>Thru Week {THRU_WEEK}</sup>',
    n_tiers=7,
    x_reversed=True,
    y_reversed=True
)

fig.update_yaxes(
    tickformat='.0%'
)
fig.update_xaxes(
    tickformat='.2f'
)
fig.update_layout_images(sizex=0.025, sizey=0.025)
fig.update_annotations(y=-.1)
fig.update_layout(width=700, height=500)

fig.show()

# Export
if EXPORT: pio.write_image(fig, f'Rushing Defense Tiers - Thru Week {THRU_WEEK}.png',scale=6, width=700, height=500)

y = 0.34819x + 0.4308107634780143
x_range = 0.403
y_range = 0.17199999
slope = 0.4267989993095398
0
0


In [18]:
''' Pass D Tiers '''

fig = tier_chart(
    data_frame=team_defense,
    y_col='Pass Success Rate',
    x_col='Pass EPA / Play',
    logos_col='team_logo_espn',
    title=f'NFL Passing Defense Tiers<br><sup>Thru Week {THRU_WEEK}</sup>',
    n_tiers=7,
    x_reversed=True,
    y_reversed=True
)


fig.update_yaxes(
    tickformat='.0%'
)
fig.update_xaxes(
    tickformat='.2f'
)
fig.update_layout_images(sizex=0.03, sizey=0.035)
fig.update_annotations(y=-.1)
fig.update_layout(width=700, height=500)

fig.show()

# Export
if EXPORT: pio.write_image(fig, f'Passing Defense Tiers - Thru Week {THRU_WEEK}.png',scale=6, width=700, height=500)

y = 0.24736x + 0.4511306436820414
x_range = 0.39000002
y_range = 0.134
slope = 0.34358975291252136
0
0


In [17]:
''' Defense Tiers '''

## Chart
fig = tier_chart(
    data_frame=team_defense,
    x_col='Pass EPA / Play',
    y_col='Rush EPA / Play',
    logos_col='team_logo_espn',
    title=f'NFL Defense Tiers<br><sup>Thru Week {THRU_WEEK}</sup>',
    n_tiers=7,
    x_reversed=True,
    y_reversed=True
)

fig.update_xaxes(
    tickformat='0.2f'
)
fig.update_yaxes(
    tickformat='0.2f'
)
fig.update_layout_images(sizey=0.06)
fig.update_annotations(y=-.1)
fig.update_layout(width=700, height=500)

fig.show()

# Export
if EXPORT: pio.write_image(fig, f'Team Defense Tiers - Thru Week {THRU_WEEK}.png',scale=6, width=700, height=500)

y = 0.23803x + -0.0902626674235212
x_range = 0.39000002
y_range = 0.403
slope = 1.0333333015441895
0
0


In [18]:
''' EPA / Snaps '''


## Create Scatter
X = team_defense['Plays'].to_numpy()
Y = team_defense['EPA / Play'].to_numpy()

scatter = px.scatter(
    x=X,
    y=Y,
    trendline='ols',
    trendline_color_override='#44546a',
    opacity=0.8
)
# Access the R-squared value
r_squared_value = px.get_trendline_results(scatter).px_fit_results.iloc[0].rsquared
print(f"R-squared value: {r_squared_value}")


## Create Figure
fig = go.Figure()

# Scatter
for trace in scatter.data:
    fig.add_trace(trace)

# Avg Lines
fig.add_vline(x=X.mean(), line_width=1, line_dash="dash", line_color="#CB4747", layer='above')
fig.add_hline(y=Y.mean(), line_width=1, line_dash="dash", line_color="#CB4747", layer='above')

# Logos
LOGOS = team_defense['team_logo_espn'].tolist()
for i in range(len(team_defense)):
    fig.add_layout_image(
        source=LOGOS[i],  # The loaded image
        xref="x",    # Reference x-coordinates to the x-axis
        yref="y",    # Reference y-coordinates to the y-axis
        x=X[i], # X-coordinate of the image's center
        y=Y[i], # Y-coordinate of the image's center
        sizex=9,   # Width of the image in data units
        sizey=9,   # Height of the image in data units
        xanchor="center", # Anchor the image by its center horizontally
        yanchor="middle", # Anchor the image by its middle vertically
        layer="above", # Place image above other plot elements
        opacity=0.9
    )

# Format
fig.update_xaxes(
    title='Snaps',
    tickformat=','
)
fig.update_yaxes(
    title='EPA / Play',
    tickformat='.2f',
    autorange='reversed'
)
fig.update_layout(
    template='nfl_template',
    title=f'Defensive EPA by Snaps<br><sup>Thru Week {THRU_WEEK}</sup>',
    width=900, height=500
)
fig.update_traces(
    marker=dict(opacity=0),
)

# Credits
fig.add_annotation(
    text=f'Figure: @clankeranalytic | Data: nfl_data_py | {datetime.today():%Y-%m-%d}',
    xref='paper', yref='paper',
    x=1, y=-.1,
    showarrow=False,
    align='right'
)

fig.show()

R-squared value: 0.24893249783404692


In [ ]:
''' Defense Ranks '''

## Data ##
defense_ranks = team_defense.copy().rename(columns={
    'RushYards': 'Rush Yards',
    'PassYards': 'Pass Yards',
    'FirstDowns': '1Ds',
    'Fumbles': 'Fumbles Rec.',
})

# Rank
data_cols = ['Plays', 'Pass Yards', 'Rush Yards', '1Ds', 'TDs', 'INTs', 'Fumbles Rec.', 'Sacks', 'TFLs', 'EPA / Play']
rank_cols = [f'{col}_rank' for col in data_cols]

for col in data_cols:
    if col in ['Pass Yards', 'Rush Yards', '1Ds', 'TDs', 'EPA / Play']:
        defense_ranks[f'{col}_rank'] = defense_ranks[col].rank(axis=0, method='max', ascending=True)
    else:
        defense_ranks[f'{col}_rank'] = defense_ranks[col].rank(axis=0, method='min', ascending=False)

defense_ranks = defense_ranks[data_cols + rank_cols]

# Logos
defense_ranks = defense_ranks.merge(team_data[['team_logo_espn', 'team_wordmark']], left_index=True, right_index=True)

print(defense_ranks.sort_values(by='EPA / Play', ascending=True).head().to_string())


## Viz ##

# Data
x = data_cols
y = defense_ranks.index.to_numpy()
z = defense_ranks[rank_cols].to_numpy()
text = defense_ranks[rank_cols].astype(int).astype(str).to_numpy()
wordmarks = defense_ranks['team_wordmark'].tolist()

# Heat map
heat_map = go.Heatmap(
    x=x, y=y, z=z, 
    text=text, texttemplate="%{text}", textfont=dict(size=10, color='white'),
    colorscale='PRGn', #['navy', '#c8c8c8', '#4bb543'], #'Cividis',
    reversescale=True,
    showscale=False,
    colorbar=dict(
        lenmode='fraction',
        len=0.5,
        title=dict(
            text='Rank',
            font=dict(weight='bold')
        ),
        xref='paper', yref='paper',
        x=1.1, xanchor='right'
    ),
    xgap=2, ygap=2
)

# Create figure
fig = go.Figure()

fig.add_trace(heat_map)

# Logos
for i in range(len(wordmarks)):
# for i in range(2):
    response = requests.get(wordmarks[i])
    logo_img = Image.open(BytesIO(response.content))
    fig.add_layout_image(
        x=-0.075,
        y=(1-(1/64))-((1/32)*i),
        sizex=.125,
        sizey=.125,
        xanchor='center',
        yanchor='middle',
        xref='paper', 
        yref='paper',
        source=logo_img,
    )
    
# Format
fig.update_xaxes(
    side='top',
    showgrid=False,
    ticks='',
    tickfont=dict(weight='bold'),
)
fig.update_yaxes(
    autorange='reversed',
    showgrid=False,
    ticks='',
    visible=False,
)
fig.update_layout(
    template='nfl_template',
    title=dict(
        text=f'NFL Defensive Ranks<br><sup>Thru Week {THRU_WEEK}</sup>',
        x=0.035
    ),
    margin=dict(t=100, b=25, l=100, r=25),
    height=900,
    width=700,
    margin_pad=5
)

# Credits
fig.add_annotation(
    text=f'Figure: @clankeranalytic | Data: nfl_data_py | {datetime.today():%Y-%m-%d}',
    xref='paper', yref='paper',
    x=1, y=-.025,
    showarrow=False,
    align='right'
)

fig.show()

# Export
if EXPORT: pio.write_image(fig, file=f'NFL Defensive Ranks - Thru Week {THRU_WEEK}.png', height=900, width=700, scale=6)

         Plays  Pass Yards  Rush Yards    1Ds   TDs  INTs  Fumbles Rec.  Sacks  TFLs  EPA / Play  Plays_rank  Pass Yards_rank  Rush Yards_rank  1Ds_rank  TDs_rank  INTs_rank  Fumbles Rec._rank  Sacks_rank  TFLs_rank  EPA / Play_rank                                     team_logo_espn                                                          team_wordmark
defteam                                                                                                                                                                                                                                                                                                                                                           
HOU        392      1249.0       619.0  112.0  14.0   8.0           3.0   16.0  13.0      -0.116        32.0              3.0              5.0       1.0       5.0        4.0               14.0        18.0       25.0              1.0  https://a.espncdn.com/i/teamlogos/nfl/500/hou.png  https

# Team

In [15]:
''' Team Tiers '''

## Data
team_df = team_offense[['EPA / Play']].merge(team_defense[['EPA / Play', 'team_logo_espn']], left_index=True, right_index=True)
team_df = team_df.rename(columns={
    'EPA / Play_x': 'Offensive EPA / Play',
    'EPA / Play_y': 'Defensive EPA / Play',
})

## Chart
fig = tier_chart(
    data_frame=team_df,
    x_col='Offensive EPA / Play',
    y_col='Defensive EPA / Play',
    logos_col='team_logo_espn',
    title=f'NFL Team Tiers<br><sup>Thru Week {THRU_WEEK}</sup>',
    n_tiers=7,
    x_reversed=False,
    y_reversed=True
)

fig.update_xaxes(
    tickformat='0.2f'
)
fig.update_yaxes(
    tickformat='0.2f'
)
fig.update_layout_images(sizey=0.04)
fig.update_annotations(y=-.1)
fig.update_layout(width=700, height=500)

fig.show()

# Export
if EXPORT: pio.write_image(fig, f'Team Tiers - Thru Week {THRU_WEEK}.png',scale=6, width=700, height=500)

y = -0.15905x + 0.0307864194166027
x_range = 0.41600000000000004
y_range = 0.294
slope = -0.7067307692307692
1
0


In [16]:
''' Week Team Tiers '''

## Data
team_df = team_offense[['EPA / Play']].merge(team_defense[['EPA / Play', 'team_logo_espn']], left_index=True, right_index=True)
team_df = team_df.rename(columns={
    'EPA / Play_x': 'Offensive EPA / Play',
    'EPA / Play_y': 'Defensive EPA / Play',
})

week_df = week_offense[['EPA / Play']].merge(week_defense[['EPA / Play', 'team_logo_espn']], left_index=True, right_index=True)
week_df = week_df.rename(columns={
    'EPA / Play_x': 'Offensive EPA / Play',
    'EPA / Play_y': 'Defensive EPA / Play',
})

total_df = pd.concat([team_df, week_df])
print(total_df.shape)
print(total_df.head().to_string())

## Chart
fig = tier_chart(
    data_frame=total_df,
    x_col='Offensive EPA / Play',
    y_col='Defensive EPA / Play',
    logos_col='team_logo_espn',
    title=f'NFL Team Tiers<br><sup>Week {THRU_WEEK}</sup>',
    n_tiers=7,
    x_reversed=False,
    y_reversed=True
)

fig.update_xaxes(
    tickformat='0.2f'
)
fig.update_yaxes(
    tickformat='0.2f'
)
fig.update_layout_images(sizey=0.12)
fig.update_annotations(y=-.1)
fig.update_layout(width=700, height=500)

for i in range(32):
    fig.layout.images[i].update(opacity=0.6, sizey=0.08)

fig.show()

# Export
if EXPORT: pio.write_image(fig, f'Team Tiers - Week {THRU_WEEK}.png',scale=6, width=700, height=500)

(58, 3)
     Offensive EPA / Play  Defensive EPA / Play                                          team_logo_espn
ARI                 0.019                 0.037       https://a.espncdn.com/i/teamlogos/nfl/500/ari.png
ATL                -0.030                 0.004       https://a.espncdn.com/i/teamlogos/nfl/500/atl.png
BAL                 0.001                 0.148       https://a.espncdn.com/i/teamlogos/nfl/500/bal.png
BUF                 0.175                -0.005       https://a.espncdn.com/i/teamlogos/nfl/500/buf.png
CAR                -0.018                 0.061  https://a.espncdn.com/i/teamlogos/nfl/500-dark/car.png
y = -0.19126x + 0.04939310801140725
x_range = 0.954
y_range = 0.954
slope = -1.0
1
0
